 - https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA

## 데이터 확인

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sub_df = pd.read_csv('sample_submission.csv')

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [6]:
#train[train.select_dtypes(include='float').columns].astype(int)

In [7]:
test.select_dtypes(include='float')

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,80.0,0.0,468.0,144.0,270.0,882.0,0.0,0.0,1961.0,1.0,730.0
1,81.0,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1958.0,1.0,312.0
2,74.0,0.0,791.0,0.0,137.0,928.0,0.0,0.0,1997.0,2.0,482.0
3,78.0,20.0,602.0,0.0,324.0,926.0,0.0,0.0,1998.0,2.0,470.0
4,43.0,0.0,263.0,0.0,1017.0,1280.0,0.0,0.0,1992.0,2.0,506.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,21.0,0.0,0.0,0.0,546.0,546.0,0.0,0.0,NaN,0.0,0.0
1455,21.0,0.0,252.0,0.0,294.0,546.0,0.0,0.0,1970.0,1.0,286.0
1456,160.0,0.0,1224.0,0.0,0.0,1224.0,1.0,0.0,1960.0,2.0,576.0
1457,62.0,0.0,337.0,0.0,575.0,912.0,0.0,1.0,NaN,0.0,0.0


### 결측치 확인

In [8]:
nan_dict = {"CNT":train.isna().sum(),
            "RATE": train.isna().sum()/train.shape[0] *100
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

# PoolQC        1453  99.520548     Pool quality                                              NA	No Pool
# MiscFeature   1406  96.301370     Miscellaneous feature not covered in other categories,    NA	None
# Alley         1369  93.767123     Type of alley access to property,                         NA 	No alley access
# Fence         1179  80.753425     Fence quality,                                            NA	No Fence
# FireplaceQu    690  47.260274     Fireplace quality                                         NA	No Fireplace
# LotFrontage    259  17.739726     Linear feet of street connected to property

# GarageType      81   5.547945     Garage location                          NA	No Garage         
# GarageYrBlt     81   5.547945     Year garage was built
# GarageFinish    81   5.547945     Interior finish of the garage            NA	No Garage
# GarageQual      81   5.547945     Garage quality                           NA	No Garage
# GarageCond      81   5.547945     Garage condition                         NA	No Garage     

# BsmtExposure    38   2.602740      Refers to walkout or garden level walls                 NA	No Basement     No	No Exposure
# BsmtFinType2    38   2.602740      Rating of basement finished area (if multiple types)    NA	No Basement
# BsmtFinType1    37   2.534247      Rating of basement finished area                        NA	No Basement
# BsmtCond        37   2.534247      Evaluates the general condition of the basement         NA	No Basement
# BsmtQual        37   2.534247      Evaluates the height of the basement                    NA	No Basement

               CNT       RATE
PoolQC        1453  99.520548
MiscFeature   1406  96.301370
Alley         1369  93.767123
Fence         1179  80.753425
FireplaceQu    690  47.260274
LotFrontage    259  17.739726
GarageType      81   5.547945
GarageYrBlt     81   5.547945
GarageFinish    81   5.547945
GarageQual      81   5.547945
GarageCond      81   5.547945
BsmtExposure    38   2.602740
BsmtFinType2    38   2.602740
BsmtFinType1    37   2.534247
BsmtCond        37   2.534247
BsmtQual        37   2.534247
MasVnrArea       8   0.547945
MasVnrType       8   0.547945
Electrical       1   0.068493


In [9]:
nan_dict = {"CNT":test.isna().sum(),
            "RATE": test.isna().sum()/test.shape[0] *100
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

# train에는 없는 null 
# MSZoning         4   0.274160            Identifies the general zoning classification of the sale.
# BsmtFullBath     2   0.137080            Basement full bathrooms
# BsmtHalfBath     2   0.137080
# Functional       2   0.137080
# Utilities        2   0.137080
# GarageCars       1   0.068540
# GarageArea       1   0.068540
# TotalBsmtSF      1   0.068540
# KitchenQual      1   0.068540
# BsmtUnfSF        1   0.068540
# BsmtFinSF2       1   0.068540
# BsmtFinSF1       1   0.068540
# Exterior2nd      1   0.068540
# Exterior1st      1   0.068540
# SaleType         1   0.068540

               CNT       RATE
PoolQC        1456  99.794380
MiscFeature   1408  96.504455
Alley         1352  92.666210
Fence         1169  80.123372
FireplaceQu    730  50.034270
LotFrontage    227  15.558602
GarageCond      78   5.346127
GarageYrBlt     78   5.346127
GarageQual      78   5.346127
GarageFinish    78   5.346127
GarageType      76   5.209047
BsmtCond        45   3.084304
BsmtExposure    44   3.015764
BsmtQual        44   3.015764
BsmtFinType1    42   2.878684
BsmtFinType2    42   2.878684
MasVnrType      16   1.096642
MasVnrArea      15   1.028101
MSZoning         4   0.274160
BsmtFullBath     2   0.137080
BsmtHalfBath     2   0.137080
Functional       2   0.137080
Utilities        2   0.137080
GarageCars       1   0.068540
GarageArea       1   0.068540
TotalBsmtSF      1   0.068540
KitchenQual      1   0.068540
BsmtUnfSF        1   0.068540
BsmtFinSF2       1   0.068540
BsmtFinSF1       1   0.068540
Exterior2nd      1   0.068540
Exterior1st      1   0.068540
SaleType  

#### 불필요한 피쳐 삭제

In [10]:
nan_features_list = nan_df[nan_df["RATE"]>50]
print(nan_features_list.index)

Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


In [11]:
train.drop(nan_features_list.index, axis=1, inplace=True)
test.drop(nan_features_list.index, axis=1, inplace=True)

In [12]:
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

In [13]:
print(train.shape, test.shape)

(1460, 75) (1459, 74)


### 상관분석

In [14]:
corr_s = train.corrwith(train["SalePrice"]).sort_values(ascending=False)
print(corr_s)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
dtype: float64


#### 불필요한 피쳐 삭제

In [15]:
corr_features_list = corr_s[abs(corr_s)<0.2]
print(corr_features_list.index)

Index(['BedroomAbvGr', 'ScreenPorch', 'PoolArea', 'MoSold', '3SsnPorch',
       'BsmtFinSF2', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
       'OverallCond', 'MSSubClass', 'EnclosedPorch', 'KitchenAbvGr'],
      dtype='object')


In [16]:
train.drop(corr_features_list.index, axis=1, inplace=True)
test.drop(corr_features_list.index, axis=1, inplace=True)

In [17]:
print(train.shape, test.shape)

(1460, 61) (1459, 60)


### 결측치 채우기

In [18]:
nan_dict = {"CNT":train.isna().sum(),
            "RATE": train.isna().sum()/train.shape[0] *100,
            "TYPE": train.dtypes
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

# null 의미 
# LotFrontage   259  17.739726  float64 --> mean 
# GarageType     81  5.547945   object  NO
# GarageYrBlt    81  5.547945  float64  Year garage was built --> 0             corr    GarageYrBlt      0.486362
# GarageFinish   81  5.547945   object  NO
# GarageQual     81  5.547945   object  NO
# GarageCond     81  5.547945   object  NO
# BsmtExposure   38  2.602740   object  NO
# BsmtFinType2   38  2.602740   object  NO
# BsmtQual       37  2.534247   object  NO
# BsmtCond       37  2.534247   object  NO
# BsmtFinType1   37  2.534247   object  NO
# MasVnrType      8  0.547945   object  Masonry veneer area in square feet  --> None 
# MasVnrArea      8  0.547945  float64  Masonry veneer type  --> 0 
# Electrical      1  0.068493   object  Electrical system  ---> 최빈값

              CNT       RATE     TYPE
LotFrontage   259  17.739726  float64
GarageType     81   5.547945   object
GarageYrBlt    81   5.547945  float64
GarageFinish   81   5.547945   object
GarageQual     81   5.547945   object
GarageCond     81   5.547945   object
BsmtExposure   38   2.602740   object
BsmtFinType2   38   2.602740   object
BsmtQual       37   2.534247   object
BsmtCond       37   2.534247   object
BsmtFinType1   37   2.534247   object
MasVnrType      8   0.547945   object
MasVnrArea      8   0.547945  float64
Electrical      1   0.068493   object


In [19]:
train[["GarageYrBlt", "MasVnrArea"]] = train[["GarageYrBlt", "MasVnrArea"]].fillna(0)
test[["GarageYrBlt", "MasVnrArea"]] = test[["GarageYrBlt", "MasVnrArea"]].fillna(0)

In [20]:
train["LotFrontage"].fillna(train["LotFrontage"].mean(), inplace=True)
test["LotFrontage"].fillna(test["LotFrontage"].mean(), inplace=True)

In [21]:
train["MasVnrType"].fillna("None", inplace=True)
test["MasVnrType"].fillna("None", inplace=True)

In [22]:
nan_dict = {"CNT":train.isna().sum(),
            "RATE": train.isna().sum()/train.shape[0] *100,
            "TYPE": train.dtypes
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

              CNT      RATE    TYPE
GarageType     81  5.547945  object
GarageFinish   81  5.547945  object
GarageQual     81  5.547945  object
GarageCond     81  5.547945  object
BsmtExposure   38  2.602740  object
BsmtFinType2   38  2.602740  object
BsmtQual       37  2.534247  object
BsmtCond       37  2.534247  object
BsmtFinType1   37  2.534247  object
Electrical      1  0.068493  object


In [23]:
train[nan_df[nan_df["RATE"]>0].index] = train[nan_df[nan_df["RATE"]>0].index].fillna("NO")
test[nan_df[nan_df["RATE"]>0].index] = test[nan_df[nan_df["RATE"]>0].index].fillna("NO")

In [24]:
nan_dict = {"CNT":train.isna().sum(),
            "RATE": train.isna().sum()/train.shape[0] *100,
            "TYPE": train.dtypes
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

Empty DataFrame
Columns: [CNT, RATE, TYPE]
Index: []


In [25]:
nan_dict = {"CNT":test.isna().sum(),
            "RATE": test.isna().sum()/test.shape[0] *100,
            "TYPE": test.dtypes
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

              CNT     RATE     TYPE
MSZoning        4  0.27416   object
Utilities       2  0.13708   object
BsmtFullBath    2  0.13708  float64
Functional      2  0.13708   object
Exterior1st     1  0.06854   object
Exterior2nd     1  0.06854   object
BsmtFinSF1      1  0.06854  float64
BsmtUnfSF       1  0.06854  float64
TotalBsmtSF     1  0.06854  float64
KitchenQual     1  0.06854   object
GarageCars      1  0.06854  float64
GarageArea      1  0.06854  float64
SaleType        1  0.06854   object


In [26]:
nan_index = nan_df[nan_df["RATE"]>0].index

In [27]:
test[nan_index] = test[nan_index].fillna(test.mode().iloc[0])
#test.dropna(inplace=True)

In [28]:
# for col in nan_index:
#     test[col].fillna(test[col].mode()[0], inplace=True)

In [29]:
nan_dict = {"CNT":test.isna().sum(),
            "RATE": test.isna().sum()/test.shape[0] *100,
            "TYPE": test.dtypes
}
nan_df = pd.DataFrame(nan_dict)
print(nan_df[nan_df["RATE"]>0].sort_values("CNT", ascending=False))

Empty DataFrame
Columns: [CNT, RATE, TYPE]
Index: []


In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 61 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1460 non-null   object 
 1   LotFrontage    1460 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   object 
 4   LotShape       1460 non-null   object 
 5   LandContour    1460 non-null   object 
 6   Utilities      1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  YearBuilt      1460 non-null   int64  
 16  YearRemodAdd   1460 non-null   int64  
 17  RoofStyle      1460 non-null   object 
 18  RoofMatl

In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 60 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1459 non-null   object 
 1   LotFrontage    1459 non-null   float64
 2   LotArea        1459 non-null   int64  
 3   Street         1459 non-null   object 
 4   LotShape       1459 non-null   object 
 5   LandContour    1459 non-null   object 
 6   Utilities      1459 non-null   object 
 7   LotConfig      1459 non-null   object 
 8   LandSlope      1459 non-null   object 
 9   Neighborhood   1459 non-null   object 
 10  Condition1     1459 non-null   object 
 11  Condition2     1459 non-null   object 
 12  BldgType       1459 non-null   object 
 13  HouseStyle     1459 non-null   object 
 14  OverallQual    1459 non-null   int64  
 15  YearBuilt      1459 non-null   int64  
 16  YearRemodAdd   1459 non-null   int64  
 17  RoofStyle      1459 non-null   object 
 18  RoofMatl

In [32]:
cols = train.select_dtypes(include='float').columns
print(cols)
train[cols] = train[cols].astype(int)



Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')


In [33]:
cols = test.select_dtypes(include='float').columns
print(cols)
test[cols] = test[cols].astype(int)

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
       'BsmtFullBath', 'GarageYrBlt', 'GarageCars', 'GarageArea'],
      dtype='object')


### Object --> Numeric

In [34]:
train.select_dtypes(include='object')

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1456,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,...,SBrkr,TA,Min1,Attchd,Unf,TA,TA,Y,WD,Normal
1457,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1458,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,FuseA,Gd,Typ,Attchd,Unf,TA,TA,Y,WD,Normal


In [35]:
object_index = train.select_dtypes(include='object').columns   # 38개

In [36]:
for col in object_index:
    print(col, train[col].unique())

MSZoning ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street ['Pave' 'Grvl']
LotShape ['Reg' 'IR1' 'IR2' 'IR3']
LandContour ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities ['AllPub' 'NoSeWa']
LotConfig ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope ['Gtl' 'Mod' 'Sev']
Neighborhood ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1 ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2 ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st ['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkF

In [37]:
print(train.shape, test.shape)

(1460, 61) (1459, 60)


In [38]:
le_encoder = LabelEncoder()

cols = object_index
for col in cols:   
    le_encoder.fit(train[col])
    train["LE_"+ col] = le_encoder.transform(train[col])
    test["LE_"+ col] = le_encoder.transform(test[col])

train.head()

,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,LE_Electrical,LE_KitchenQual,LE_Functional,LE_GarageType,LE_GarageFinish,LE_GarageQual,LE_GarageCond,LE_PavedDrive,LE_SaleType,LE_SaleCondition
0,RL,65,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,5,2,6,1,2,5,5,2,8,4
1,RL,80,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,5,3,6,1,2,5,5,2,8,4
2,RL,68,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,5,2,6,1,2,5,5,2,8,4
3,RL,60,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,5,2,6,5,3,5,5,2,8,0
4,RL,84,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,5,2,6,1,2,5,5,2,8,4


In [39]:
test.head()

,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,LE_Electrical,LE_KitchenQual,LE_Functional,LE_GarageType,LE_GarageFinish,LE_GarageQual,LE_GarageCond,LE_PavedDrive,LE_SaleType,LE_SaleCondition
0,RH,80,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,5,3,6,1,3,5,5,2,8,4
1,RL,81,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,5,2,6,1,3,5,5,2,8,4
2,RL,74,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,5,3,6,1,0,5,5,2,8,4
3,RL,78,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,5,2,6,1,0,5,5,2,8,4
4,RL,43,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,...,5,2,6,1,2,5,5,2,8,4


In [40]:
train.drop(object_index, axis=1, inplace=True)
test.drop(object_index, axis=1, inplace=True)

In [41]:
print(train.shape, test.shape)

(1460, 61) (1459, 60)


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 61 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   LotFrontage       1460 non-null   int32
 1   LotArea           1460 non-null   int64
 2   OverallQual       1460 non-null   int64
 3   YearBuilt         1460 non-null   int64
 4   YearRemodAdd      1460 non-null   int64
 5   MasVnrArea        1460 non-null   int32
 6   BsmtFinSF1        1460 non-null   int64
 7   BsmtUnfSF         1460 non-null   int64
 8   TotalBsmtSF       1460 non-null   int64
 9   1stFlrSF          1460 non-null   int64
 10  2ndFlrSF          1460 non-null   int64
 11  GrLivArea         1460 non-null   int64
 12  BsmtFullBath      1460 non-null   int64
 13  FullBath          1460 non-null   int64
 14  HalfBath          1460 non-null   int64
 15  TotRmsAbvGrd      1460 non-null   int64
 16  Fireplaces        1460 non-null   int64
 17  GarageYrBlt       1460 non-null  

In [43]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   LotFrontage       1459 non-null   int32
 1   LotArea           1459 non-null   int64
 2   OverallQual       1459 non-null   int64
 3   YearBuilt         1459 non-null   int64
 4   YearRemodAdd      1459 non-null   int64
 5   MasVnrArea        1459 non-null   int32
 6   BsmtFinSF1        1459 non-null   int32
 7   BsmtUnfSF         1459 non-null   int32
 8   TotalBsmtSF       1459 non-null   int32
 9   1stFlrSF          1459 non-null   int64
 10  2ndFlrSF          1459 non-null   int64
 11  GrLivArea         1459 non-null   int64
 12  BsmtFullBath      1459 non-null   int32
 13  FullBath          1459 non-null   int64
 14  HalfBath          1459 non-null   int64
 15  TotRmsAbvGrd      1459 non-null   int64
 16  Fireplaces        1459 non-null   int64
 17  GarageYrBlt       1459 non-null  

#### 불필요한 피쳐 삭제

In [44]:
corr_s = train.corrwith(train["SalePrice"]).sort_values(ascending=False)
print(corr_s[abs(corr_s)<0.2])

LE_HouseStyle      0.180163
LE_RoofMatl        0.132383
LE_ExterCond       0.117303
LE_Functional      0.115328
LE_Exterior2nd     0.103766
LE_Exterior1st     0.103551
LE_BsmtCond        0.091503
LE_Condition1      0.091155
LE_BsmtFinType2    0.072717
LE_LandSlope       0.051152
LE_Street          0.041036
LE_MasVnrType      0.018216
LE_LandContour     0.015453
LE_Condition2      0.007513
LE_Utilities      -0.014314
LE_SaleType       -0.054911
LE_LotConfig      -0.067396
LE_BldgType       -0.085591
LE_BsmtFinType1   -0.098734
LE_Heating        -0.098812
LE_MSZoning       -0.166872
dtype: float64


In [45]:
corr_features_list = corr_s[abs(corr_s)<0.2]

train.drop(corr_features_list.index, axis=1, inplace=True)
test.drop(corr_features_list.index, axis=1, inplace=True)
print(train.shape, test.shape)

(1460, 40) (1459, 39)


In [46]:
corr_s = train.corrwith(train["SalePrice"]).sort_values(ascending=False)
print(corr_s)

SalePrice           1.000000
OverallQual         0.790982
GrLivArea           0.708624
GarageCars          0.640409
GarageArea          0.623431
TotalBsmtSF         0.613581
1stFlrSF            0.605852
FullBath            0.560664
TotRmsAbvGrd        0.533723
YearBuilt           0.522897
YearRemodAdd        0.507101
MasVnrArea          0.472614
Fireplaces          0.466929
BsmtFinSF1          0.386420
LE_Foundation       0.382479
LotFrontage         0.334897
WoodDeckSF          0.324413
2ndFlrSF            0.319334
OpenPorchSF         0.315856
HalfBath            0.284108
LotArea             0.263843
GarageYrBlt         0.261366
LE_CentralAir       0.251328
LE_GarageCond       0.246705
LE_Electrical       0.237044
LE_PavedDrive       0.231357
BsmtFullBath        0.227122
LE_RoofStyle        0.222405
BsmtUnfSF           0.214479
LE_SaleCondition    0.213092
LE_Neighborhood     0.210851
LE_GarageQual       0.205963
LE_LotShape        -0.255580
LE_BsmtExposure    -0.294589
LE_HeatingQC  

### Scaler

In [47]:
train.columns

Index(['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'SalePrice', 'LE_LotShape',
       'LE_Neighborhood', 'LE_RoofStyle', 'LE_ExterQual', 'LE_Foundation',
       'LE_BsmtQual', 'LE_BsmtExposure', 'LE_HeatingQC', 'LE_CentralAir',
       'LE_Electrical', 'LE_KitchenQual', 'LE_GarageType', 'LE_GarageFinish',
       'LE_GarageQual', 'LE_GarageCond', 'LE_PavedDrive', 'LE_SaleCondition'],
      dtype='object')

In [48]:
train.iloc[:, 0:23].describe()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,70.041096,10516.828082,6.099315,1971.267808,1984.865753,103.117123,443.639726,567.240411,1057.429452,1162.626712,...,1.565068,0.382877,6.517808,0.613014,1868.739726,1.767123,472.980137,94.244521,46.660274,180921.195890
std,22.024031,9981.264932,1.382997,30.202904,20.645407,180.731373,456.098091,441.866955,438.705324,386.587738,...,0.550916,0.502885,1.625393,0.644666,453.697295,0.747315,213.804841,125.338794,66.256028,79442.502883
min,21.000000,1300.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,334.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,60.000000,7553.500000,5.000000,1954.000000,1967.000000,0.000000,0.000000,223.000000,795.750000,882.000000,...,1.000000,0.000000,5.000000,0.000000,1958.000000,1.000000,334.500000,0.000000,0.000000,129975.000000
50%,70.000000,9478.500000,6.000000,1973.000000,1994.000000,0.000000,383.500000,477.500000,991.500000,1087.000000,...,2.000000,0.000000,6.000000,1.000000,1977.000000,2.000000,480.000000,0.000000,25.000000,163000.000000
75%,79.000000,11601.500000,7.000000,2000.000000,2004.000000,164.250000,712.250000,808.000000,1298.250000,1391.250000,...,2.000000,1.000000,7.000000,1.000000,2001.000000,2.000000,576.000000,168.000000,68.000000,214000.000000
max,313.000000,215245.000000,10.000000,2010.000000,2010.000000,1600.000000,5644.000000,2336.000000,6110.000000,4692.000000,...,3.000000,2.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,755000.000000


In [49]:
# fig, axes = plt.subplots(ncols=8, nrows=3, figsize=(30,20))
# for i, feature in enumerate(train.columns):
#     cols = i%8
#     rows = i//8     
#     sns.distplot(train[feature], kde=True, ax=axes[rows][cols])
#     axes[rows][cols].set_title(feature)

In [50]:
scale_features = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'GarageYrBlt',  'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'SalePrice']
for sf in scale_features:
    train[sf] = np.log1p(train[sf])
train.head()    

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,LE_HeatingQC,LE_CentralAir,LE_Electrical,LE_KitchenQual,LE_GarageType,LE_GarageFinish,LE_GarageQual,LE_GarageCond,LE_PavedDrive,LE_SaleCondition
0,4.189655,9.042040,7,7.602900,7.602900,5.283204,6.561031,5.017280,6.753438,6.753438,...,0,1,5,2,1,2,5,5,2,4
1,4.394449,9.169623,6,7.589336,7.589336,0.000000,6.886532,5.652489,7.141245,7.141245,...,0,1,5,3,1,2,5,5,2,4
2,4.234107,9.328212,7,7.601902,7.602401,5.093750,6.188264,6.075346,6.825460,6.825460,...,0,1,5,2,1,2,5,5,2,4
3,4.110874,9.164401,7,7.557995,7.586296,0.000000,5.379897,6.293419,6.629363,6.869014,...,2,1,5,2,5,3,5,5,2,0
4,4.442651,9.565284,8,7.601402,7.601402,5.860786,6.486161,6.196444,7.044033,7.044033,...,0,1,5,2,1,2,5,5,2,4


In [51]:
scale_features = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'GarageYrBlt',  'GarageArea',
       'WoodDeckSF', 'OpenPorchSF']
for sf in scale_features:
    test[sf] = np.log1p(test[sf])
test.head()    

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,LE_HeatingQC,LE_CentralAir,LE_Electrical,LE_KitchenQual,LE_GarageType,LE_GarageFinish,LE_GarageQual,LE_GarageCond,LE_PavedDrive,LE_SaleCondition
0,4.394449,9.360741,5,7.581720,7.581720,0.000000,6.150603,5.602119,6.783325,6.799056,...,4,1,5,3,1,3,5,5,2,4
1,4.406719,9.565775,6,7.580189,7.580189,4.691348,6.828712,6.008813,7.192934,7.192934,...,4,1,5,2,1,3,5,5,2,4
2,4.317488,9.534668,5,7.599902,7.600402,0.000000,6.674561,4.927254,6.834109,6.834109,...,2,1,5,3,1,0,5,5,2,4
3,4.369448,9.208238,6,7.600402,7.600402,3.044522,6.401917,5.783825,6.831954,6.831954,...,0,1,5,2,1,0,5,5,2,4
4,3.784190,8.518392,8,7.597396,7.597396,0.000000,5.575949,6.925595,7.155396,7.155396,...,0,1,5,2,1,2,5,5,2,4


In [52]:
corr_s = train.corrwith(train["SalePrice"]).sort_values(ascending=False)
print(corr_s)

SalePrice           1.000000
OverallQual         0.817185
GrLivArea           0.730254
GarageCars          0.680625
1stFlrSF            0.608955
FullBath            0.594771
YearBuilt           0.584443
YearRemodAdd        0.565590
TotRmsAbvGrd        0.534422
Fireplaces          0.489450
OpenPorchSF         0.459948
GarageArea          0.454898
MasVnrArea          0.413318
LE_Foundation       0.412083
LotArea             0.399923
TotalBsmtSF         0.372838
LE_CentralAir       0.351599
LotFrontage         0.346651
WoodDeckSF          0.343039
GarageYrBlt         0.326485
LE_GarageCond       0.323691
HalfBath            0.313982
LE_PavedDrive       0.304687
LE_Electrical       0.297400
LE_GarageQual       0.275692
LE_SaleCondition    0.248819
BsmtFullBath        0.236224
BsmtUnfSF           0.208233
BsmtFinSF1          0.208099
LE_Neighborhood     0.201685
LE_RoofStyle        0.186917
2ndFlrSF            0.180778
LE_BsmtExposure    -0.277372
LE_LotShape        -0.278091
LE_GarageFinis

In [53]:
corr_features_list = corr_s[abs(corr_s)<0.2]
print(corr_features_list.index)

Index(['LE_RoofStyle', '2ndFlrSF'], dtype='object')


In [54]:
train.drop(corr_features_list.index, axis=1, inplace=True)
test.drop(corr_features_list.index, axis=1, inplace=True)

In [55]:
print(train.shape, test.shape)

(1460, 38) (1459, 37)


In [56]:
print(train.columns)
print(test.columns)

Index(['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'SalePrice', 'LE_LotShape', 'LE_Neighborhood',
       'LE_ExterQual', 'LE_Foundation', 'LE_BsmtQual', 'LE_BsmtExposure',
       'LE_HeatingQC', 'LE_CentralAir', 'LE_Electrical', 'LE_KitchenQual',
       'LE_GarageType', 'LE_GarageFinish', 'LE_GarageQual', 'LE_GarageCond',
       'LE_PavedDrive', 'LE_SaleCondition'],
      dtype='object')
Index(['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'LE_

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LotFrontage       1460 non-null   float64
 1   LotArea           1460 non-null   float64
 2   OverallQual       1460 non-null   int64  
 3   YearBuilt         1460 non-null   float64
 4   YearRemodAdd      1460 non-null   float64
 5   MasVnrArea        1460 non-null   float64
 6   BsmtFinSF1        1460 non-null   float64
 7   BsmtUnfSF         1460 non-null   float64
 8   TotalBsmtSF       1460 non-null   float64
 9   1stFlrSF          1460 non-null   float64
 10  GrLivArea         1460 non-null   float64
 11  BsmtFullBath      1460 non-null   int64  
 12  FullBath          1460 non-null   int64  
 13  HalfBath          1460 non-null   int64  
 14  TotRmsAbvGrd      1460 non-null   int64  
 15  Fireplaces        1460 non-null   int64  
 16  GarageYrBlt       1460 non-null   float64


In [58]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LotFrontage       1459 non-null   float64
 1   LotArea           1459 non-null   float64
 2   OverallQual       1459 non-null   int64  
 3   YearBuilt         1459 non-null   float64
 4   YearRemodAdd      1459 non-null   float64
 5   MasVnrArea        1459 non-null   float64
 6   BsmtFinSF1        1459 non-null   float64
 7   BsmtUnfSF         1459 non-null   float64
 8   TotalBsmtSF       1459 non-null   float64
 9   1stFlrSF          1459 non-null   float64
 10  GrLivArea         1459 non-null   float64
 11  BsmtFullBath      1459 non-null   int32  
 12  FullBath          1459 non-null   int64  
 13  HalfBath          1459 non-null   int64  
 14  TotRmsAbvGrd      1459 non-null   int64  
 15  Fireplaces        1459 non-null   int64  
 16  GarageYrBlt       1459 non-null   float64


In [59]:
# cols = train.select_dtypes(include='float').columns
# print(cols)
# train[cols] = train[cols].astype(int)

## 점수확인

### columns 37

In [88]:
X_df = train.drop(["SalePrice"], axis=1)
y_df = train["SalePrice"]

X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, shuffle=True, random_state = 121)
print(X_df.shape, y_df.shape)
print(X_train.shape, y_train.shape)

(1460, 37) (1460,)
(1168, 37) (1168,)


In [90]:
# LinearRegression()
lasso = Lasso(alpha=1.0)    # a*|W|
ridge = Ridge(alpha=1.0)    # a*W^2
elsnet = ElasticNet(alpha=1.0, l1_ratio=0.2)  # Lasso + Ridge      # 0.2(a*|W|) + 0.8(a*W^2)

In [91]:
models = [lasso, ridge, elsnet]
for model in models:
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    mse = mean_squared_error(y_val, pred, squared=True)
    rmse = mean_squared_error(y_val, pred, squared=False)
    print(f'{model.__class__.__name__} MSE:{mse:.5f} RMSE:{rmse:.5f}') 
    
# Lasso MSE:0.15270 RMSE:0.39077
# Ridge MSE:0.01522 RMSE:0.12338
# ElasticNet MSE:0.07162 RMSE:0.26761

Lasso MSE:0.15270 RMSE:0.39077
Ridge MSE:0.01522 RMSE:0.12338
ElasticNet MSE:0.07161 RMSE:0.26761


In [92]:
cv_model = GridSearchCV(ridge, param_grid={'alpha':[0.01, 0.05, 1.0, 5.0, 10]},
            scoring='neg_root_mean_squared_error',
            cv = 10, refit=True)
cv_model.fit(X_train, y_train)
print(f'RMSE: {cv_model.best_score_ * -1:.5f}')
print(cv_model.best_params_)

RMSE: 0.14496
{'alpha': 0.01}


In [93]:
pred = cv_model.predict(X_val)  
#print(pred)
mse = mean_squared_error(y_val, pred, squared=True)
rmse = mean_squared_error(y_val, pred, squared=False)
print(f'MSE:{mse:.5f} RMSE:{rmse:.5f}')

# MSE:0.01502 RMSE:0.12257

MSE:0.01502 RMSE:0.12257


In [94]:
pred = cv_model.predict(test)

In [100]:
print(pred) #sclaer된 pred

predo = np.round(np.expm1(pred), 6)
print(predo)

[11.620085   11.98279801 12.10407545 ... 11.946353   11.70698952
 12.37566468]
[111310.181942 159978.027716 180605.416378 ... 154252.556559 121416.405497
 236963.276358]


In [101]:
sub_df

,Id,SalePrice
0,1461,111310.2
1,1462,159978.0
2,1463,180605.4
3,1464,196287.1
4,1465,199137.4
...,...,...
1454,2915,81867.9
1455,2916,83855.3
1456,2917,154252.6
1457,2918,121416.4


In [102]:
print(sub_df.shape)
print(len(predo))

(1459, 2)
1459


In [103]:
sub_df["SalePrice"] = np.array(predo).reshape(-1, 1)

In [104]:
sub_df

,Id,SalePrice
0,1461,111310.181942
1,1462,159978.027716
2,1463,180605.416378
3,1464,196287.084582
4,1465,199137.423449
...,...,...
1454,2915,81867.928705
1455,2916,83855.296475
1456,2917,154252.556559
1457,2918,121416.405497


In [105]:
sub_df.to_csv("gcv_submission.csv", index=False)

### columns 28

In [106]:
corr_s = train.corrwith(train["SalePrice"]).sort_values(ascending=False)
print(corr_s)

SalePrice           1.000000
OverallQual         0.817185
GrLivArea           0.730254
GarageCars          0.680625
1stFlrSF            0.608955
FullBath            0.594771
YearBuilt           0.584443
YearRemodAdd        0.565590
TotRmsAbvGrd        0.534422
Fireplaces          0.489450
OpenPorchSF         0.459948
GarageArea          0.454898
MasVnrArea          0.413318
LE_Foundation       0.412083
LotArea             0.399923
TotalBsmtSF         0.372838
LE_CentralAir       0.351599
LotFrontage         0.346651
WoodDeckSF          0.343039
GarageYrBlt         0.326485
LE_GarageCond       0.323691
HalfBath            0.313982
LE_PavedDrive       0.304687
LE_Electrical       0.297400
LE_GarageQual       0.275692
LE_SaleCondition    0.248819
BsmtFullBath        0.236224
BsmtUnfSF           0.208233
BsmtFinSF1          0.208099
LE_Neighborhood     0.201685
LE_BsmtExposure    -0.277372
LE_LotShape        -0.278091
LE_GarageFinish    -0.417405
LE_HeatingQC       -0.425906
LE_GarageType 

In [107]:
corr_features_list = corr_s[abs(corr_s)<0.3]
print(corr_features_list.index)

Index(['LE_Electrical', 'LE_GarageQual', 'LE_SaleCondition', 'BsmtFullBath',
       'BsmtUnfSF', 'BsmtFinSF1', 'LE_Neighborhood', 'LE_BsmtExposure',
       'LE_LotShape'],
      dtype='object')


In [108]:
train.drop(corr_features_list.index, axis=1, inplace=True)
test.drop(corr_features_list.index, axis=1, inplace=True)

print(train.shape, test.shape)

(1460, 29) (1459, 28)


In [109]:
X_df = train.drop(["SalePrice"], axis=1)
y_df = train["SalePrice"]

X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, shuffle=True, random_state = 121)
print(X_df.shape, y_df.shape)
print(X_train.shape, y_train.shape)

(1460, 28) (1460,)
(1168, 28) (1168,)


In [110]:
cv_model = GridSearchCV(ridge, param_grid={'alpha':[0.01, 0.05, 1.0, 5.0, 10]},
            scoring='neg_root_mean_squared_error',
            cv = 10, refit=True)
cv_model.fit(X_train, y_train)
print(f'RMSE: {cv_model.best_score_ * -1:.5f}')
print(cv_model.best_params_)

pred = cv_model.predict(X_val)  
#print(pred)
mse = mean_squared_error(y_val, pred, squared=True)
rmse = mean_squared_error(y_val, pred, squared=False)
print(f'MSE:{mse:.5f} RMSE:{rmse:.5f}')

# RMSE: 0.15108
# {'alpha': 0.01}
# MSE:0.01786 RMSE:0.13365

RMSE: 0.15108
{'alpha': 0.01}
MSE:0.01786 RMSE:0.13365


### Bagging

In [ ]:
bagg_ridge = BaggingRegressor(base_estimator=ridge(),
                        n_estimators=10, random_state=0).fit(X, y)

## PCA 

In [72]:
pca = PCA(n_components=20)
#pipeline = make_pipeline(std_scaler, pca)
pca_res = pca.fit_transform(X_df)
print(pca_res.shape)
print("주성분(PC) 15개가 전체 데이터를 얼마나 설명할수 있는가?\n", np.sum(pca.explained_variance_ratio_), pca.explained_variance_ratio_)


(1460, 20)
주성분(PC) 15개가 전체 데이터를 얼마나 설명할수 있는가?
 0.9772745527517893 [0.36608169 0.14601292 0.09561096 0.05957306 0.0523731  0.045483
 0.0333506  0.02707455 0.02360717 0.0219927  0.01948587 0.0171504
 0.01587964 0.01236559 0.01092841 0.00760484 0.00696326 0.00644551
 0.00524583 0.00404545]


In [73]:
pca_df = pd.DataFrame(data=pca_res)
pca_df["SalePrice"] = y_df
pca_df
# 어떠한 피쳐가 합쳐진건지는 알 수 없음 

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,SalePrice
0,-6.403456,4.335117,-0.935582,-4.259310,1.425046,-1.059251,1.295180,0.354490,0.520906,1.181259,...,-0.940888,-0.040510,0.180748,0.081780,0.693316,-0.624469,0.116114,-0.324611,0.209371,12.247699
1,11.621584,-0.262670,-2.515405,4.138186,-1.958079,0.098838,-1.268268,-2.710972,-0.043410,0.405664,...,-1.535169,0.204201,1.019702,-0.404453,-0.089372,-0.357597,0.626959,-0.481578,-0.835130,12.109016
2,-6.469757,4.585131,-0.821865,-3.724892,1.093503,-1.185786,0.935500,-1.131622,-0.338602,-0.646130,...,1.570340,0.089950,0.360454,-0.060808,0.627333,-0.615644,-0.092110,0.322225,-0.149818,12.317171
3,-6.244895,-0.907297,-0.934411,-1.909513,-0.804699,-2.758099,-0.838745,2.092558,1.530341,0.750538,...,2.514639,-0.505796,0.842826,3.595342,0.057294,-0.113313,-1.278918,0.758452,-0.176617,11.849405
4,3.862227,6.663298,0.135270,0.628349,1.484787,0.319100,0.150896,0.319076,1.283738,-0.167503,...,0.695392,0.505193,1.633276,-0.508740,0.362617,-0.699861,-0.114731,0.223168,-0.463397,12.429220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-4.287198,-1.029671,4.570405,-2.070399,-1.846118,-2.297529,0.762749,-0.540861,-0.394219,-0.516223,...,-1.078542,-0.472310,0.533837,0.009588,0.165547,-0.353807,0.448422,-0.968861,-0.487617,12.072547
1456,2.068178,2.571759,-2.416705,1.116929,-0.558243,3.834775,-2.686292,1.248195,0.259258,-0.353608,...,-1.358844,-0.336348,-1.092131,-0.265580,0.940710,-0.847202,0.507392,-0.410764,-0.331285,12.254868
1457,-5.799380,1.636977,0.476287,-2.325241,-0.489232,-4.395331,-0.359923,0.212393,1.866931,0.153229,...,-1.693687,-0.140800,0.859354,-0.223877,-0.251144,-1.218182,-0.864572,1.459515,0.545310,12.493133
1458,-0.814219,-1.713518,-2.630103,4.648078,-2.456041,2.428252,2.925577,-0.231693,-0.615382,-0.072206,...,-1.239788,4.522860,0.125261,-0.779116,0.381952,0.727087,-2.438040,-1.892688,1.083206,11.864469


In [74]:
pca_df_y = pca_df["SalePrice"]  #df["traget"]  
pca_df_X = pca_df[pca_df.columns.difference(["SalePrice"])]
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(pca_df_X, pca_df_y, test_size=0.2,  random_state=36, shuffle=True)

models = [lasso, ridge, elsnet]
for model in models:
    model.fit(X_train_pca, y_train_pca)
    pred = model.predict(X_val_pca)
    mse = mean_squared_error(y_val_pca, pred, squared=True)
    rmse = mean_squared_error(y_val_pca, pred, squared=False)
    print(f'{model.__class__.__name__} MSE:{mse:.5f} RMSE:{rmse:.5f}') 

# n = 20
# Lasso MSE:0.18575 RMSE:0.43099
# Ridge MSE:0.02982 RMSE:0.17268
# ElasticNet MSE:0.07380 RMSE:0.27166

# n = 15    
# Lasso MSE:0.18575 RMSE:0.43099
# Ridge MSE:0.03514 RMSE:0.18745
# ElasticNet MSE:0.07380 RMSE:0.27166

# n = 10
# Lasso MSE:0.18575 RMSE:0.43099
# Ridge MSE:0.03728 RMSE:0.19307
# ElasticNet MSE:0.07380 RMSE:0.27166

Lasso MSE:0.18575 RMSE:0.43099
Ridge MSE:0.02982 RMSE:0.17268
ElasticNet MSE:0.07380 RMSE:0.27166
